In [1]:
import requests
import sys
import gzip
import csv
import os
import pandas as pd
from tempfile import TemporaryDirectory 
from pathlib import Path, PurePath
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import pip

In [14]:
pip.main(['install', 'scrapy'])
pip.main(['install', 'itemadapter'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


0

In [2]:
output = Path('.').resolve().parents[0].joinpath('module_6/input', 'links.csv')
sitemap_link='https://auto.ru/sitemapindex.xml'

In [3]:
temp_fold = TemporaryDirectory()
temp_path = Path(temp_fold.name)
r = requests.get(sitemap_link)
text = r.content.decode()
soup = BeautifulSoup(text, 'lxml')
links_to_gz = []
i = 0
for element in soup.find_all('loc'):
    links_to_gz.append(element.text.strip())
    i += 1
print(f'Found {i} link lists as gzipped xml files.')

Found 125 link lists as gzipped xml files.


In [4]:
offers_cars = [x for x in links_to_gz if 'offers_cars' in x]

In [5]:
i=0
for link in tqdm(offers_cars):
    tempfile = temp_path / link.split('/')[-1]
    if not tempfile.is_file():
        r = requests.get(link, allow_redirects=True)
        with tempfile.open('wb') as file:
            file.write(r.content)
        i+=1
print(f'Downloaded {i} link files')


Downloaded 11 link files


In [6]:
i = 0
for gz in tqdm(list(temp_path.iterdir())):
    gz = gz.resolve()
    if gz.suffix == '.gz':
        with gzip.open(gz) as archive:
            with open(gz.parent / gz.stem, 'wb') as output_file:
                output_file.write(archive.read())
        gz.unlink()
    i += 1
print(f'Unzipped {i} txt files.')


Unzipped 11 txt files.


In [7]:

offers_list = []
for txt in tqdm(list(temp_path.iterdir())):
    offers_counter = 0
    with open(txt, 'r') as f:
        file_content = f.read().splitlines()
        offers_list += file_content
        print(f'Found {len(file_content)} links in file {txt}, totally {len(offers_list)} links')
# write this list to csv
output_path = Path(output).resolve()
with output_path.open('w+', newline='') as f:
    wr = csv.writer(f)
    for line in offers_list:
        wr.writerow([line])
print(f'Output written to file {str(output_path)}')
# cleanup temp folder
temp_fold.cleanup()

Found 300004 links in file C:\Users\ALLKOT~1\AppData\Local\Temp\tmph26a88hb\sitemap_offers_cars_1.xml, totally 300004 links
Found 300004 links in file C:\Users\ALLKOT~1\AppData\Local\Temp\tmph26a88hb\sitemap_offers_cars_10.xml, totally 600008 links
Found 252340 links in file C:\Users\ALLKOT~1\AppData\Local\Temp\tmph26a88hb\sitemap_offers_cars_11.xml, totally 852348 links
Found 300004 links in file C:\Users\ALLKOT~1\AppData\Local\Temp\tmph26a88hb\sitemap_offers_cars_2.xml, totally 1152352 links
Found 300004 links in file C:\Users\ALLKOT~1\AppData\Local\Temp\tmph26a88hb\sitemap_offers_cars_3.xml, totally 1452356 links
Found 300004 links in file C:\Users\ALLKOT~1\AppData\Local\Temp\tmph26a88hb\sitemap_offers_cars_4.xml, totally 1752360 links
Found 300004 links in file C:\Users\ALLKOT~1\AppData\Local\Temp\tmph26a88hb\sitemap_offers_cars_5.xml, totally 2052364 links
Found 300004 links in file C:\Users\ALLKOT~1\AppData\Local\Temp\tmph26a88hb\sitemap_offers_cars_6.xml, totally 2352368 links
F

In [13]:
bashCommand = """%%bash
cd .. &&
touch './input/dataset_with_electro.csv' &&
output_file_path=$(realpath ./input/dataset_with_electro.csv) &&
links_file=$(realpath ./input/links.csv) && 
cd ./auto_ru_scrapy && 
scrapy crawl -a links_file=$links_file -o $output_file_path -L ERROR auto_ru_scraper"""
import subprocess
process = subprocess.Popen(bashCommand, stdout=subprocess.PIPE, shell=True)
output, error = process.communicate()

In [18]:
input_path = Path('.').resolve().parents[0].joinpath('module_6/input')
test_links = pd.read_csv(input_path.joinpath('test.csv'), usecols=['car_url'])
test_links.to_csv(input_path.joinpath('test_links.csv'), header = False, index = False)

In [ ]:
%%bash
cd .. &&
touch './input/test_scraped.csv' &&
output_file_path=$(realpath ./input/test_scraped.csv) &&
links_file=$(realpath ./input/test_links.csv) && 
cd ./auto_ru_scrapy && 
scrapy crawl -a links_file=$links_file -o $output_file_path -L ERROR auto_ru_scraper